In [16]:
import plotly.express as px
import pandas as pd
from google.cloud import bigquery
import numpy as np
from datetime import time, timedelta, datetime
import math 
import warnings
warnings.filterwarnings('ignore')
import scipy.stats as st
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import os
import ipywidgets
import statsmodels.stats.weightstats as sw
from statsmodels.stats.proportion import test_proportions_2indep
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
from google.cloud import bigquery
from scipy.stats import norm
from scipy import stats
pd.options.display.float_format = "{:,.2f}".format
from scipy.stats import norm
from scipy import stats
import statsmodels.stats.weightstats as ws
import openpyxl

# df["Week"] = df["Date"].dt.to_period("W").dt.to_timestamp()

bigquery_client = bigquery.Client(project='analytics-dev-333113')

def cycle_sql(start, end, query, weeks=False):
    """
    You have to use {day} in your script to add cycle date into this backets
    """
    date_start = datetime.strptime(start, '%Y-%m-%d')
    date_end = datetime.strptime(end, '%Y-%m-%d')

    if weeks == False:
        daterange = [(date_start + timedelta(days=x)).strftime('%Y-%m-%d') for x in range(((date_end-date_start).days)+1)]
    else:
        daterange = [(date_start + timedelta(weeks=x)).strftime('%Y-%m-%d') for x in range(((date_end-date_start).days//7)+1)] # weeks dividing days by 7

    total_df = pd.DataFrame()

    for date in daterange:
        print(f"Uploading - {date}:", datetime.today().strftime('%Y-%m-%d %H:%M:%S'))
        script = query.format(date = date)
        df_cycle = bigquery_client.query(script).to_dataframe()
        if df_cycle.empty == True:
            print('Dataframe is empty')
        total_df = pd.concat([df_cycle, total_df])
    return total_df

def read_bq(query, project='analytics-dev-333113'):
    client = bigquery.Client(project=project)
    query_job = client.query(query)
    result_df = query_job.to_dataframe()
    return result_df



def writing_excel(name:str, dataset1=None, dataset2=None, dataset3=None, dataset4=None):
    with pd.ExcelWriter(f"{name}.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index 
    # to store the dataframe in specified sheet

        if dataset1 is not None:
            if dataset2 is not None:
                if dataset3 is not None:
                    if dataset4 is not None:
                        dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                        #   index=False
                                          )
                        dataset2.to_excel(writer, sheet_name=f"2-{name}", 
                                        #   index=False
                                          )
                        dataset3.to_excel(writer, sheet_name=f"3-{name}", 
                                        #   index=False
                                          )
                        dataset4.to_excel(writer, sheet_name=f"4-{name}", 
                                        #   index=False
                                          )
                    else:
                        dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                        #   index=False
                                          )
                        dataset2.to_excel(writer, sheet_name=f"2-{name}", 
                                        #   index=False
                                          )
                        dataset3.to_excel(writer, sheet_name=f"3-{name}", 
                                        #   index=False
                                          )
                else:
                    dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                    #   index=False
                                      )
                    dataset2.to_excel(writer, sheet_name=f"2-{name}", 
                                    #   index=False
                                      )
            else:
                dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                #   index=False
                                  )

        print('DataFrame is written to Excel File successfully.')





# Event table checking on PHC using swrve table

In [42]:
df_checking_os = read_bq("""
SELECT event_dt_part,
       os_name || ' / ' || (JSON_EXTRACT_SCALAR(payload, '$.photocontrol_type') || ' / ' || name) segment,
       COUNT(user_id) AS                                                                          cnt
FROM indriver-e6e40.emart.swrve_event
WHERE 1 = 1
  AND event_dt_part > '2024-01-01'
  AND name IN (
               'city.driver.photocontrol_banner.show',
               'city.driver.photocontrol_banner.click',
               'city.driver.photocontrol_screen.show',
               'city.driver.photocontrol_photo_confirm.click',
               'city.driver.photocontrol_photo_cancel.show',
               'city.driver.photocontrol_photo_cancel.click',
               'city.driver.photocontrol_photo_upload_success.show',
               'city.driver.photocontrol_apply_after_declined.click'
    )
  AND JSON_EXTRACT_SCALAR(payload, '$.photocontrol_type') IN ('auto', 'avatar')
GROUP BY 1, 2;
""")

df_checking_os

,event_dt_part,segment,cnt
0,2024-08-07,ios / avatar / city.driver.photocontrol_banner...,344
1,2024-10-06,ios / auto / city.driver.photocontrol_banner.show,123588
2,2024-10-17,ipados / auto / city.driver.photocontrol_banne...,6
3,2024-08-01,ios / avatar / city.driver.photocontrol_photo_...,1
4,2024-07-10,android / auto / city.driver.photocontrol_bann...,34070
...,...,...,...
2370,2024-09-01,ipados / avatar / city.driver.photocontrol_ban...,2
2371,2024-09-03,android / auto / city.driver.photocontrol_bann...,1313278
2372,2024-05-05,android / avatar / city.driver.photocontrol_ba...,204093
2373,2024-08-09,android / avatar / city.driver.photocontrol_ba...,747455


In [43]:
df_checking_os.groupby('segment', as_index=False)['cnt'].agg('sum').sort_values(['segment', 'cnt'], ascending=True).set_index('segment')

,cnt
segment,
android / auto / city.driver.photocontrol_banner.click,8655646
android / auto / city.driver.photocontrol_banner.show,217493782
android / avatar / city.driver.photocontrol_banner.click,11721304
android / avatar / city.driver.photocontrol_banner.show,129409370
ios / auto / city.driver.photocontrol_banner.click,656559
ios / auto / city.driver.photocontrol_banner.show,11520713
ios / auto / city.driver.photocontrol_photo_confirm.click,158
ios / auto / city.driver.photocontrol_screen.show,304
ios / avatar / city.driver.photocontrol_banner.click,746782


In [45]:
fig = px.line(df_checking_os.sort_values(['segment', 'event_dt_part'], ascending=True)
            , x="event_dt_part"
            , y='cnt'
            , color='segment'
            , width=1400, height=600
            )

fig.show()


# 'city.driver.photocontrol_banner.show', -- has android on avatar & auto
# 'city.driver.photocontrol_banner.click', -- has android on avatar & auto
# 'city.driver.photocontrol_screen.show', -- has just IOS since september 
# 'city.driver.photocontrol_photo_confirm.click', -- has just IOS since september on avatar & auto
# 'city.driver.photocontrol_photo_cancel.show', -- there is nothing on avatar & auto
# 'city.driver.photocontrol_photo_cancel.click', -- there is nothing on avatar & auto
# 'city.driver.photocontrol_photo_upload_success.show', -- has IOS just on avatar
# 'city.driver.photocontrol_apply_after_declined.click' -- there is nothing on avatar & auto

# Back structure checking on PHC using dwh-storage-327422.ods_photocontrol_cdc.events table

In [13]:
df_checking_os_back = read_bq("""
WITH cte AS (SELECT DISTINCT e.check_uuid,
                             user_id,
                             created_date_part
             FROM `dwh-storage-327422.ods_photocontrol_cdc.events` AS e
             WHERE e.created_date_part >= '2024-06-01'
               AND e.type = "TYPE_AUTO"
               AND SOURCE = 'facechecker'),
     cte2 AS (SELECT id,
                     created,
                     mode,
                     osversion,
                     devicemodel
              FROM indriver-e6e40.ods_monolith.tbl_user)
SELECT created_date_part,
       CASE
           WHEN osversion LIKE '%android%' THEN 'android'
           WHEN osversion LIKE '%ios%' THEN 'ios'
           ELSE NULL
           END os,
       COUNT(t1.user_id)
FROM cte t1
         JOIN cte2 t2
              ON t1.user_id = t2.id
GROUP BY 1, 2
""")

df_checking_os_back

,created_date_part,os,f0_
0,2024-06-26,android,46305
1,2024-10-07,ios,11903
2,2024-07-10,android,38142
3,2024-09-24,android,68416
4,2024-10-02,android,69354
...,...,...,...
276,2024-10-04,android,68171
277,2024-08-11,ios,11859
278,2024-07-06,android,37977
279,2024-08-26,ios,17642


In [15]:
fig = px.line(df_checking_os_back.sort_values('created_date_part', ascending=True)
            , x="created_date_part"
            , y='f0_'
            , color='os'
            , width=1400, height=600
            )

fig.show()